In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sksurv.util import Surv
from sksurv.ensemble import GradientBoostingSurvivalAnalysis, RandomSurvivalForest
from sksurv.metrics import concordance_index_censored
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from category_encoders import TargetEncoder
from lifelines import CoxPHFitter

In [ ]:
data_full = pd.read_excel(r'data_22_ready.xlsx', sheet_name='Лист1')

In [ ]:
# 2. Преобразование целевой переменной
y_full = Surv.from_dataframe(event='risk_event', time='duration', data=data_full)
y_event_full = y_full['risk_event'] 
y_time_full = y_full['duration']
X_full = data_full.drop(['risk_event', 'duration'], axis=1)

In [ ]:
X_full['sport'] = X_full['sport'].replace({5: 0})
X_full['sport'] = X_full['sport'].replace({3: 2})
X_full['sport'] = X_full['sport'].replace({4: 2})

In [ ]:
y_fd = pd.DataFrame(y_full)

In [ ]:
# 3. Определение типов признаков Модель (1)
numeric_features = ['age', 'bmi', 'income', 'h_number']
nominal_low_cardinality = ['gender', 'hospitalizations', 'preventive_measures', 'cardiovascular', 'endocrine', 'respiratory', 'gastrointestinal', 'musculoskeletal', 'neurological', 'allergy', 'smoker', 'alcohol', 'marital_status', 'children', 'place_type', 'empl_status']          # Номинальные (мало категорий)
ordinal_features = ['sport', 'health_self_assessment', 'education', 'life_satisfaction']        # Порядковые

In [ ]:
# 3. Определение типов признаков Модель (2)
numeric_features = ['age', 'h_number', 'income', 'bmi']
nominal_low_cardinality = ['hospitalizations', 'gender', 'preventive_measures', 'cardiovascular', 'neurological', 'endocrine', 'musculoskeletal', 'smoker', 'alcohol', 'marital_status', 'children', 'place_type', 'empl_status']          # Номинальные (мало категорий)
ordinal_features = ['sport', 'education', 'life_satisfaction']        # Порядковые

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted

# Пайплайн обработки данных
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
        ]), numeric_features),
        
        ('nominal_low', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
        ]), nominal_low_cardinality),

        ('ordinal', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
        ]), ordinal_features)
    ],
    remainder='drop',
    verbose_feature_names_out=False
)

In [ ]:
# FULL DATA
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import Pipeline
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import concordance_index_censored
from sksurv.util import Surv

# Стратифицированное разделение данных
X_train, X_test, y_train, y_test = train_test_split(
    X_full,
    y_full,
    test_size=0.4,
    stratify=y_full["risk_event"],  # Стратификация по событию
    random_state=42
)


# Пайплайн с RSF
rsf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomSurvivalForest(
        n_estimators=200,
        min_samples_split=10,
        min_samples_leaf=5,
        random_state=42
    ))
])

# Кастомная функция для оценки C-index
def score_model(model, X_full, y_full):
    pred = model.predict(X_full)
    return concordance_index_censored(y_full['risk_event'], y_full['duration'], pred)[0]

# Стратифицированная кросс-валидация
def stratified_cv_survival(model, X_full, y_full, n_splits=5):
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    fold_scores = []
    
    for train_idx, test_idx in cv.split(X_full, y_full["risk_event"]):
        # Разделение данных
        X_train_fold = X_full.iloc[train_idx]
        X_test_fold = X_full.iloc[test_idx]
        y_train_fold = y_full[train_idx]
        y_test_fold = y_full[test_idx]
        
        # Обучение и предсказание
        model.fit(X_train_fold, y_train_fold)
        score = score_model(model, X_test_fold, y_test_fold)
        fold_scores.append(score)
        
    return np.mean(fold_scores), np.std(fold_scores)

# Запуск кросс-валидации
mean_cv_score, std_cv_score = stratified_cv_survival(
    rsf_pipeline, 
    X_train, 
    y_train,
    n_splits=5
)

print(f"Cross-validated C-index: {mean_cv_score:.3f} ± {std_cv_score:.3f}")

# Финальное обучение на всех тренировочных данных
rsf_pipeline.fit(X_train, y_train)

# Оценка на тестовом наборе
test_preds = rsf_pipeline.predict(X_test)
test_cindex = concordance_index_censored(
    y_test["risk_event"], 
    y_test["duration"], 
    test_preds
)[0]

print(f"\nTest C-index: {test_cindex:.3f}")

In [ ]:
# RSF Full data
import numpy as np
from sksurv.metrics import integrated_brier_score
survival_funcs = rsf_pipeline.predict_survival_function(X_test)

# Максимальное время в тестовых данных (уменьшенное на 1e-6)
max_test_time = np.max(y_test["duration"]) - 1e-6
min_test_time = np.min(y_test["duration"])

# Генерация временных точек в допустимом диапазоне
times = np.linspace(min_test_time, max_test_time, 10)

# Преобразование StepFunction в числовые вероятности
probabilities = np.array([[fn(t) for t in times] for fn in survival_funcs])

# Вычисление Integrated Brier Score
ibs = integrated_brier_score(
    y_train, 
    y_test, 
    probabilities, 
    times
)

print(f"Integrated Brier Score: {ibs:.3f}")

In [ ]:
X_processed = preprocessor.transform(X_full) # изменение для target encoder, было y_full["risk_event"]

# Объединение обработанных данных с временем и статусом
processed_data = pd.DataFrame(
    X_processed,
    columns=preprocessor.get_feature_names_out()
)
processed_data = pd.concat([processed_data, X_full["id"], y_pd["risk_event"], y_pd["duration"]], axis=1)

In [ ]:
# Предсказание функций выживания для всех наблюдений
surv_funcs = rsf_pipeline.predict_survival_function(X_full, return_array=False)

# Группировка
smoker_indices = processed_data[processed_data["smoker"] == 1].index
non_smoker_indices = processed_data[processed_data["smoker"] == 0].index

# Усреднение кривых
time_points = np.linspace(0, processed_data["duration"].max(), 100)
mean_smokers = np.mean([func(time_points) for func in surv_funcs[smoker_indices]], axis=0)
mean_non_smokers = np.mean([func(time_points) for func in surv_funcs[non_smoker_indices]], axis=0)

# Визуализация
plt.figure(figsize=(10, 6))
plt.step(time_points, mean_smokers, where="post", color="red", label="Курильщики")
plt.step(time_points, mean_non_smokers, where="post", color="green", label="Некурящие")
plt.xlabel("Время")
plt.ylabel("Вероятность выживания")
plt.title("Кривые выживаемости (RSF) для курильщиков и некурящих")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Full Data, все итоговые признаки по Cox (2)
model = rsf_pipeline.named_steps['model']
preprocessor = rsf_pipeline.named_steps['preprocessor']
import shap
from sklearn.base import BaseEstimator

# Создание обертки для RSF, чтобы SHAP мог работать с моделью
class RSFWrapper(BaseEstimator):
    def __init__(self, model):
        self.model = model

    def predict(self, X, y=None):
        return self.model.predict(X)  # Возвращает прогнозируемый риск

# Инициализация обертки
rsf_wrapped = RSFWrapper(model)

# Предобработка данных
X_test_preprocessed = preprocessor.transform(X_test)

# Создание SHAP-объяснителя
explainer = shap.Explainer(rsf_wrapped.predict, X_test_preprocessed)
shap_values = explainer(X_test_preprocessed)

# Визуализация важности признаков
feature_names = preprocessor.get_feature_names_out()
shap.summary_plot(shap_values, X_test_preprocessed, feature_names=feature_names)

In [ ]:
from lifelines.utils import concordance_index
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
import math
def add_significance_stars(p_value):
    if p_value < 0.01:
        return "***"
    elif p_value < 0.05:
        return "**"
    elif p_value < 0.1:
        return "*"
    else:
        return ""

In [ ]:
# Модель Кокса (1)
y_pd=pd.DataFrame(y_full)
# Модель Cox PH
# Предобработка данных
#X["age_time"] = X["age"] * y["duration"]
X_processed = preprocessor.fit_transform(X_full) # изменение для target encoder, было y_full["risk_event"]

# Объединение обработанных данных с временем и статусом
processed_data = pd.DataFrame(
    X_processed,
    columns=preprocessor.get_feature_names_out()
)
processed_data = pd.concat([processed_data, y_pd["risk_event"], y_pd["duration"]], axis=1)

# Обучение модели Cox PH
model_cox_1 = CoxPHFitter()
model_cox_1.fit(processed_data, duration_col="duration", event_col="risk_event")

In [ ]:
summary_df = model_cox_1.summary
summary_df["significance"] = summary_df["p"].apply(add_significance_stars)
print(summary_df[["coef", "exp(coef)", "p", "significance"]])

In [ ]:
# Итог полная модель Кокса (1)
log_likelihood = model_cox_1.log_likelihood_
print(f"Log Partial Likelihood: {log_likelihood:.4f}")
aic = model_cox_1.AIC_partial_
print(f"AIC: {aic:.2f}")
model_cox_1.print_summary() 

In [ ]:
# Модель Кокса (2)
y_pd=pd.DataFrame(y_full)
# Модель Cox PH
# Предобработка данных
#X["age_time"] = X["age"] * y["duration"]
X_processed = preprocessor.fit_transform(X_full) # изменение для target encoder, было y_full["risk_event"]

# Объединение обработанных данных с временем и статусом
processed_data = pd.DataFrame(
    X_processed,
    columns=preprocessor.get_feature_names_out()
)
processed_data = pd.concat([processed_data, y_pd["risk_event"], y_pd["duration"]], axis=1)

# Обучение модели Cox PH
model_cox_2 = CoxPHFitter()
model_cox_2.fit(processed_data, duration_col="duration", event_col="risk_event")

In [ ]:
summary_df = model_cox_2.summary
summary_df["significance"] = summary_df["p"].apply(add_significance_stars)
print(summary_df[["coef", "exp(coef)", "p", "significance"]])

In [ ]:
# Группировка по курильщикам и некурящим
smokers = processed_data[processed_data["smoker"] == 1]
non_smokers = processed_data[processed_data["smoker"] == 0]

model_pred_survival = model_cox_2.predict_survival_function(processed_data)
# Усреднение кривых внутри групп
mean_survival_smokers = model_pred_survival[smokers.index].mean(axis=1)
mean_survival_non_smokers = model_pred_survival[non_smokers.index].mean(axis=1)

# Визуализация
plt.figure(figsize=(10, 6))
plt.plot(mean_survival_smokers, color="red", label="Курильщики")
plt.plot(mean_survival_non_smokers, color="green", label="Некурящие")
plt.xlabel("Время")
plt.ylabel("Вероятность выживания")
plt.title("Кривые выживания (Cox PH) для курильщиков и некурящих")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Группировка по полу
men = processed_data[processed_data["gender"] == 1]
women = processed_data[processed_data["gender"] == 0]

# Усреднение кривых внутри групп
mean_survival_men = model_pred_survival[men.index].mean(axis=1)
mean_survival_women = model_pred_survival[women.index].mean(axis=1)

# Визуализация
plt.figure(figsize=(10, 6))
plt.plot(mean_survival_men, color="red", label="Мужчины")
plt.plot(mean_survival_women, color="green", label="Женщины")
plt.xlabel("Время")
plt.ylabel("Вероятность выживания")
plt.title("Кривые выживания (Cox PH) для мужчин и женщин")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Группировка по с/с заболеваниям
cardio = processed_data[processed_data["cardiovascular"] == 1]
non_cardio = processed_data[processed_data["cardiovascular"] == 0]

# Усреднение кривых внутри групп
mean_survival_cardio = model_pred_survival[cardio.index].mean(axis=1)
mean_survival_non_cardio = model_pred_survival[non_cardio.index].mean(axis=1)

# Визуализация
plt.figure(figsize=(10, 6))
plt.plot(mean_survival_cardio, color="red", label="Есть сердечно-сосудистые заболевания")
plt.plot(mean_survival_non_cardio, color="green", label="Сердечно-сосудистые заболевания отсутствуют")
plt.xlabel("Время")
plt.ylabel("Вероятность выживания")
plt.title("Кривые выживания (Cox PH) с группировкой по наличию сердечно-сосудистых заболеваний")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Итог сокращенная модель Кокса (2)
log_likelihood = model_cox_2.log_likelihood_
print(f"Log Partial Likelihood: {log_likelihood:.4f}")
aic = model_cox_2.AIC_partial_
print(f"AIC: {aic:.2f}")
model_cox_2.print_summary()

In [ ]:
from lifelines.statistics import proportional_hazard_test

# Проверка предположения
test_results = proportional_hazard_test(model_cox_1, new_X, time_transform="rank")
print(test_results.summary)

In [ ]:
summary_df.to_excel("cox_2_results.xlsx", index=False)  # Excel

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
model_cox_2.plot()
#plt.axvline(x=1, color='grey', linestyle='--')  # HR=1 (нейтральный эффект)
plt.title("Коэффициенты модели Cox PH (2) с 95% доверительными интервалами")
plt.show()

In [ ]:
numeric_features = ['age', 'bmi', 'income']
nominal_low_cardinality = ['gender', 'hospitalizations', 'preventive_measures', 'cardiovascular', 'smoker', 'alcohol']          # Номинальные (мало категорий)
#nominal_high_cardinality = ['region', 'sport', 'employment_industry']       # Номинальные (много категорий)
ordinal_features = ['income_satisfaction', 'education', 'life_satisfaction']        # Порядковые

In [ ]:
#new_X = X_full.drop(columns=['bmi', 'hospitalizations', 'endocrine', 'respiratory', 'gastrointestinal', 'musculoskeletal', 'neurological', 'allergy', 'alcohol', 'sport', 'health_self_assessment', 'doctor_visit_per_year'])
X_processed = preprocessor.transform(X_full)
processed_data = pd.DataFrame(
    X_processed,
    columns=preprocessor.get_feature_names_out()
)
#processed_data=pd.concat([processed_data, y_pd["risk_event"], y_pd["duration"]], axis=1)

In [ ]:
new_X = processed_data.drop(columns=['bmi', 'endocrine', 'respiratory', 'gastrointestinal', 'musculoskeletal', 'neurological', 'allergy', 'alcohol', 'sport', 'health_self_assessment', 'doctor_visit_per_year', 'h_number', 'income', 'income_per_person', 'income_satisfaction', 'life_satisfaction', 'marital_status', 'med_satisfaction'])

In [ ]:
array_np = new_X.to_numpy()

In [ ]:
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import integrated_brier_score
from sksurv.util import Surv
import numpy as np

# 1. Подготовка данных
# Пример создания структурированного массива y_train и y_test
X_train, X_test, y_train, y_test = train_test_split(
    processed_data,
    y_full,
    test_size=0.4,
    stratify=y_full["risk_event"],  # Стратификация по событию
    random_state=42
)
y_train = Surv.from_arrays(event=y_train['risk_event'], time=y_train['duration'])
y_test = Surv.from_arrays(event=y_test['risk_event'], time=y_test['duration'])

# 2. Обучение модели
model = CoxPHSurvivalAnalysis()
model.fit(X_train, y_train)

# 3. Предсказание функций выживания
surv_funcs = model.predict_survival_function(X_test)

# 4. Временные точки для расчета IBS
# Используйте стандартные имена полей 'event' и 'time' для доступа к данным
times_train = np.unique(y_train[y_train['event']]['time'])
max_test_time = np.max(y_test['time'])  # Максимальное время в тестовой выборке
times = times_train[times_train < max_test_time]  # Фильтруем времена

# Если times пуст после фильтрации, берем времена из тестовых данных
if len(times) == 0:
    times = np.unique(y_test[y_test['event']]['time'])

# 5. Преобразование в массив numpy
# Убедитесь, что surv_funcs содержит функции, а не числа
surv_pred = np.array([fn(times) for fn in surv_funcs])

# 6. Расчет Integrated Brier Score
ibs = integrated_brier_score(
    y_train, 
    y_test, 
    surv_pred, 
    times
)
print(f"Integrated Brier Score: {ibs:.3f}")

In [ ]:
# Cox (2)
c_index = model.concordance_index_
print(f"C-index: {c_index:.3f}")
# Значение от 0.5 (случайность) до 1 (идеальное предсказание).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from lifelines import KaplanMeierFitter
from statsmodels.nonparametric.kernel_regression import KernelReg
from scipy.interpolate import CubicSpline

model_rsf = rsf_pipeline.named_steps['model']
models = {
    "CoxPH": model_cox_2,
    "RandomSurvivalForest": model_rsf
}

# Общая временная сетка
max_duration = y_full['duration'].max()
common_times = np.linspace(0, max_duration, 100)

survival_curves = {}
X_processed = preprocessor.transform(X_full)
X_processed = pd.DataFrame(
    X_processed,
    columns=preprocessor.get_feature_names_out()
)


for name, model in models.items():
    if name == "RandomSurvivalForest":
        survival_fns = model.predict_survival_function(X_processed)
        
        interpolated_curves = []
        for fn in survival_fns:
            model_times = fn.x
            model_survival = fn(model_times)
            interp_func = interp1d(
                model_times, 
                model_survival, 
                kind='linear', 
                fill_value=(1.0, 0.0), 
                bounds_error=False
            )
            interpolated_curves.append(interp_func(common_times))
        
        survival_curves[name] = np.array(interpolated_curves)
    
    elif name == "CoxPH":
        # Явное добавление t=0
        #common_times_with_zero = np.concatenate([[0], common_times])
        survival = model.predict_survival_function(X_processed, times=common_times).T
        # Убедимся, что S(0)=1
        survival.iloc[:, 0] = 1.0
        survival_curves[name] = survival
        # Транспонируем матрицу кривых выживаемости
        #survival_curves[name] = model.predict_survival_function(
            #X_processed, 
            #times=common_times
        #).T

# Инициализация и расчет Каплана-Мейера
kmf = KaplanMeierFitter()
kmf.fit(y_full['duration'], y_full['risk_event'])
# Получение данных
km_ci = kmf.confidence_interval_
km_survival = kmf.survival_function_at_times(common_times)
km_ci_lower = kmf.confidence_interval_['KM_estimate_lower_0.95']
km_ci_upper = kmf.confidence_interval_['KM_estimate_upper_0.95']

# Проверка размерностей
for name, curves in survival_curves.items():
    print(f"{name}: curves.shape = {curves.shape}")

# Построение графиков
plt.figure(figsize=(12, 6))
for name, curves in survival_curves.items():
    mean_survival = curves.mean(axis=0)  # Теперь усреднение по правильной оси
    plt.plot(common_times, mean_survival, label=name, lw=2)

# Кривая Каплана-Мейера с доверительным интервалом
#plt.step(common_times, km_survival, where='post', 
         #label='Kaplan-Meier', linestyle='--', color='black')

times = kmf.timeline
survival = kmf.survival_function_.values.flatten()
# Интерполяция
cs = CubicSpline(times, survival, bc_type='natural')
smooth_times = np.linspace(times.min(), times.max(), 100)
smooth_survival = cs(smooth_times)

# Построение графиков
plt.plot(smooth_times, smooth_survival, label='Kaplan-Meier', color='black')

# Заливка доверительного интервала
plt.fill_between(
    km_ci.index, 
    km_ci_lower, 
    km_ci_upper, 
    color='gray', 
    alpha=0.3, 
    label='95% доверительный интервал'
)

plt.xlabel("Время в годах", fontsize=12)
plt.ylabel("Средняя вероятность выживания", fontsize=12)
plt.title("Сравнение средних кривых выживания", fontsize=14)
plt.grid(alpha=0.3)
plt.legend()
plt.show()

In [ ]:
X_processed = preprocessor.transform(X_full)
X_processed = pd.DataFrame(
    X_processed,
    columns=preprocessor.get_feature_names_out()
)

In [ ]:
# Временная сетка от 1 до 10 лет
times = np.arange(0, 10)

# Предсказание выживаемости
survival_cox = model_cox_2.predict_survival_function(X_processed.iloc[[126]], times=times)
S_cox = survival_cox.values.flatten()  # S(t | X)

In [ ]:
survival_rsf = model_rsf.predict_survival_function(X_processed)
from scipy.interpolate import interp1d

# Временные точки, для которых нужно получить значения
target_times = np.arange(0, 10)

# Интерполяция для каждого наблюдения
interpolated_survival = []
for curve in survival_rsf:
    # Кривая выживаемости для текущего наблюдения
    model_times = curve.x
    model_survival = curve(model_times)
    # Линейная интерполяция
    interp_func = interp1d(
        model_times, 
        model_survival, 
        kind="linear", 
        fill_value=(1.0, 0.0), 
        bounds_error=False
    )
    interpolated_survival.append(interp_func(target_times))

# Преобразование в массив
S_rsf = np.array(interpolated_survival)

In [ ]:
def calculate_death_prob(S):
    P = np.zeros(10)
    P[0] = 1 - S[0]  # Для t=1: P(T=1) = 1 - S(1)
    for t in range(1, 10):
        P[t] = S[t-1] - S[t]
    return P

P_cox = calculate_death_prob(S_cox)
P_rsf = calculate_death_prob(S_rsf[126])

In [ ]:
# id=92 (индекс 15)
i = 0.1579
v = 1 / (1 + i)
discount_factors = np.array([v**t for t in range(1, 11)])
C = 100_000
premium_cox = C * np.sum(P_cox * discount_factors)
premium_rsf = C * np.sum(P_rsf * discount_factors)
print(f"Премия (Cox PH): {premium_cox:.2f} руб.")
print(f"Премия (Random Survival Forest): {premium_rsf:.2f} руб.")

In [ ]:
# id=119 (индекс 24)
i = 0.1579
v = 1 / (1 + i)
discount_factors = np.array([v**t for t in range(1, 11)])
C = 100_000
premium_cox = C * np.sum(P_cox * discount_factors)
premium_rsf = C * np.sum(P_rsf * discount_factors)
print(f"Премия (Cox PH): {premium_cox:.2f} руб.")
print(f"Премия (Random Survival Forest): {premium_rsf:.2f} руб.")

In [ ]:
# id=606 (индекс 108)
i = 0.1579
v = 1 / (1 + i)
discount_factors = np.array([v**t for t in range(1, 11)])
C = 100_000
premium_cox = C * np.sum(P_cox * discount_factors)
premium_rsf = C * np.sum(P_rsf * discount_factors)
print(f"Премия (Cox PH): {premium_cox:.2f} руб.")
print(f"Премия (Random Survival Forest): {premium_rsf:.2f} руб.")

In [ ]:
# id=645 (индекс 126)
i = 0.1579
v = 1 / (1 + i)
discount_factors = np.array([v**t for t in range(1, 11)])
C = 100_000
premium_cox = C * np.sum(P_cox * discount_factors)
premium_rsf = C * np.sum(P_rsf * discount_factors)
print(f"Премия (Cox PH): {premium_cox:.2f} руб.")
print(f"Премия (Random Survival Forest): {premium_rsf:.2f} руб.")

In [ ]:
def calculate_premium(model, X_processed, model_type):
    # Временные точки (1-10 лет)
    times = np.arange(1, 11)
    discount_factors = np.array([(1 / (1 + 0.1579))**t for t in times])
    C = 100_000

    # Получение кривых выживаемости
    if model_type == "CoxPH":
        survival = model.predict_survival_function(X_processed, times=times).values.T
    elif model_type == "RSF":
        survival = []
        for individual in X_processed.iterrows():
            curve = model.predict_survival_function(individual[1].to_frame().T)
            interpolated = interp1d(
                curve[0].x, 
                curve[0].y, 
                kind="linear", 
                fill_value=(1.0, 0.0), 
                bounds_error=False
            )(times)
            survival.append(interpolated)
        survival = np.array(survival)

    # Расчет премий для всех индивидов
    premiums = []
    for S in survival:
        P = np.zeros(10)
        P[0] = 1 - S[0]
        for t in range(1, 10):
            P[t] = S[t-1] - S[t]
        premium = C * np.sum(P * discount_factors)
        premiums.append(premium)

    return np.sum(premiums)

# Расчет суммарной премии для Cox PH
total_premium_cox = calculate_premium(model_cox_2, X_processed, "CoxPH")

# Расчет суммарной премии для RSF
total_premium_rsf = calculate_premium(model_rsf, X_processed, "RSF")

print(f"Суммарная премия (Cox PH): {total_premium_cox:.2f} руб.")
print(f"Суммарная премия (RSF): {total_premium_rsf:.2f} руб.")

In [ ]:
(21781242.34 - 21516970.21)/11179

In [ ]:
import numpy as np
import pandas as pd
times = np.arange(0, 10)
y_pd=pd.DataFrame(y_full)
X_processed = preprocessor.transform(X_full) # изменение для target encoder, было y_full["risk_event"]

# Объединение обработанных данных с временем и статусом
processed_data = pd.DataFrame(
    X_processed,
    columns=preprocessor.get_feature_names_out()
)
processed_data = pd.concat([processed_data, X_full["id"], y_pd["risk_event"], y_pd["duration"]], axis=1)

In [ ]:
from sksurv.ensemble import RandomSurvivalForest
sample = processed_data[processed_data['id'].isin([92])]

# Предположим, что модель `rsf` уже обучена
surv_probs_rsf = rsf_pipeline.predict_survival_function(sample, return_array=True)  # Массив формы (1, n_times)

# Преобразуем в DataFrame/Series
surv_rsf = pd.Series(surv_probs_rsf[0], index=times)

# Построение графика
#plt.step(surv_rsf.index, surv_rsf.values, label='RSF')

#plt.xlabel('Время')
#plt.ylabel('Вероятность выживания')
#plt.legend()
#plt.show()

In [ ]:
# Пример с lifelines
from lifelines import CoxPHFitter

# Обучение модели
cph = CoxPHFitter()
cph.fit(processed_data, duration_col='duration', event_col='risk_event', formula='age + hospitalizations + alcohol + place_type + sport + empl_status + gender + marital_status + preventive_measures + cardiovascular + smoker + neurological + education + life_satisfaction + h_number + bmi + income + endocrine + children + musculoskeletal')

# Предсказание для индивида с заданными признаками
times = model_cox_2.baseline_survival_.index  # Временные точки из базовой кривой

# Кривая выживаемости
survival_curve = model_cox_2.predict_survival_function(processed_data[processed_data['id'].isin([92])], times=times)

In [ ]:
# 92
plt.step(surv_rsf.index, surv_rsf.values, label='RSF')
plt.step(times, survival_curve, label='Cox PH')  # Или surv_cox.squeeze()
plt.xlabel('Время')
plt.ylabel('Вероятность выживания')
plt.legend()
plt.show()

In [ ]:
# 119
plt.step(surv_rsf.index, surv_rsf.values, label='RSF')
plt.step(times, survival_curve, label='Cox PH')  # Или surv_cox.squeeze()
plt.xlabel('Время')
plt.ylabel('Вероятность выживания')
plt.legend()
plt.show()